Our goal is to download kline data for a given pair. Most of the code is taken from the
github repo https://github.com/pratikpv/cryptocurrency_data_downloader. 

We use the code below for 

In [1]:
from binance.client import Client
import os
import pandas as pd
from cryptocurrency_data_downloader.download_data_from_binance import *

In [2]:
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')
client = Client(api_key, api_secret)

In [3]:
org_columns = ['open',
               'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
               'trades', 'tb_base_av', 'tb_quote_av', 'ignore']

fmt = "%Y-%m-%d %H:%M:%S"

In [4]:
def download_data_from_binance(symbol, client,
                                  from_date, to_date, output_filename, step=0, pause=-1, simulate=False,):
    """

    :param symbol:
    :param from_date:
    :param to_date:
    :param output_filename:
    :param step: step in number of days. Download data in batches of days given by 'step'
    :param pause: pause seconds before downloading next batch.
        if pause == -1 --> random sleep(2,5)
        if pause == 0 --> no sleep
        if pause == num--> sleep for num of seconds
    :param simulate:
    :return:
    """
    
    #binance_client = init_mod()
    from_date_obj = datetime.strptime(from_date, fmt)
    step_date_obj = from_date_obj + timedelta(days=step)
    step_date = step_date_obj.strftime(fmt)

    from_millis = to_unixmillis(from_date)
    to_millis = to_unixmillis(to_date)
    step_millis = to_unixmillis(step_date)

    count = 0
    while True:
        from_millis_str = str(from_millis)
        step_millis_str = str(step_millis)
        print('Step %d:Downloading data from %s to %s' % (count,
                                                          str(to_datetime(from_millis_str)),
                                                          str(to_datetime(step_millis_str))
                                                          ))
        if not simulate:
            # download data

            klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE,
                                                          from_millis_str, end_str=step_millis_str)
            klines_len = len(klines)
            if klines_len == 0:
                print('\t Failed to download from %s to %s. Got %d' % (str(to_datetime(from_millis_str)),
                                                                       str(to_datetime(step_millis_str)), klines_len
                                                                       ))
                time.sleep(5)

            print('\t Downloaded data of len %d from %s to %s' % (klines_len,
                                                                  str(to_datetime(from_millis_str)),
                                                                  str(to_datetime(step_millis_str))
                                                                  ))
            new_columns = [item + '_' + symbol for item in org_columns]
            new_columns.insert(0, 'timestamp')

            data_df = pd.DataFrame(klines,
                                   columns=new_columns)
            data_df['timestamp'] = pd.to_datetime(data_df['timestamp'], unit='ms')
            data_df.set_index('timestamp', inplace=True)
            data_df.to_csv(output_filename,mode = 'a',header = False)

        # move to next step of batches
        from_millis = step_millis
        step_date_obj = step_date_obj + timedelta(days=step)
        step_date = step_date_obj.strftime(fmt)
        step_millis = to_unixmillis(step_date)
        count = count + 1
        if pause == -1:
            pause = np.random.randint(2, 5)
        time.sleep(pause)
        if step_millis >= to_millis:
            break

In [5]:
from_date = '2020-05-05 00:00:00'
to_date = '2021-12-03 00:00:00'
output_filename = 'ETH_USDT3.csv'
step = 5
pause = 5
symbol = 'ETHUSDT'

In [6]:
download_data_from_binance(symbol, client,from_date, to_date, output_filename, step, pause, simulate=False)



Step 0:Downloading data from 2020-05-05 09:00:00 to 2020-05-10 09:00:00
	 Downloaded data of len 7201 from 2020-05-05 09:00:00 to 2020-05-10 09:00:00
Step 1:Downloading data from 2020-05-10 09:00:00 to 2020-05-15 09:00:00


KeyboardInterrupt: 

In [25]:
#from_date = '2019-06-01 00:00:00'
#to_date = '2020-01-01 00:00:00'
from_date = '2020-06-08 00:00:00'
to_date = '2021-02-06 00:00:00'
output_filename = 'ADAUSDT_1minute_kline'
step = 5
pause = 5
symbol = 'ADAUSDT'

In [26]:
download_data_from_binance(symbol, client,from_date, to_date, output_filename, step, pause, simulate=False)


Step 0:Downloading data from 2020-06-08 09:00:00 to 2020-06-13 09:00:00
	 Downloaded data of len 7201 from 2020-06-08 09:00:00 to 2020-06-13 09:00:00
Step 1:Downloading data from 2020-06-13 09:00:00 to 2020-06-18 09:00:00
	 Downloaded data of len 7201 from 2020-06-13 09:00:00 to 2020-06-18 09:00:00
Step 2:Downloading data from 2020-06-18 09:00:00 to 2020-06-23 09:00:00
	 Downloaded data of len 7201 from 2020-06-18 09:00:00 to 2020-06-23 09:00:00
Step 3:Downloading data from 2020-06-23 09:00:00 to 2020-06-28 09:00:00
	 Downloaded data of len 6991 from 2020-06-23 09:00:00 to 2020-06-28 09:00:00
Step 4:Downloading data from 2020-06-28 09:00:00 to 2020-07-03 09:00:00
	 Downloaded data of len 7201 from 2020-06-28 09:00:00 to 2020-07-03 09:00:00
Step 5:Downloading data from 2020-07-03 09:00:00 to 2020-07-08 09:00:00
	 Downloaded data of len 7201 from 2020-07-03 09:00:00 to 2020-07-08 09:00:00
Step 6:Downloading data from 2020-07-08 09:00:00 to 2020-07-13 09:00:00
	 Downloaded data of len 720

In [28]:
download_data_from_binance(symbol, client,from_date, to_date, output_filename, step, pause, simulate=False)

In [31]:
#from_date = '2019-06-01 00:00:00'
#to_date = '2020-01-01 00:00:00'
from_date = '2018-10-27 00:00:00'
to_date = '2021-02-06 00:00:00'
output_filename = 'BNBUSDT_1minute_kline'
step = 5
pause = 5
symbol = 'BNBUSDT'

In [32]:
download_data_from_binance(symbol, client,from_date, to_date, output_filename, step, pause, simulate=False)

Step 0:Downloading data from 2018-10-27 09:00:00 to 2018-11-01 09:00:00
	 Downloaded data of len 7201 from 2018-10-27 09:00:00 to 2018-11-01 09:00:00
Step 1:Downloading data from 2018-11-01 09:00:00 to 2018-11-06 09:00:00
	 Downloaded data of len 7201 from 2018-11-01 09:00:00 to 2018-11-06 09:00:00
Step 2:Downloading data from 2018-11-06 09:00:00 to 2018-11-11 09:00:00
	 Downloaded data of len 7201 from 2018-11-06 09:00:00 to 2018-11-11 09:00:00
Step 3:Downloading data from 2018-11-11 09:00:00 to 2018-11-16 09:00:00
	 Downloaded data of len 6781 from 2018-11-11 09:00:00 to 2018-11-16 09:00:00
Step 4:Downloading data from 2018-11-16 09:00:00 to 2018-11-21 09:00:00
	 Downloaded data of len 7201 from 2018-11-16 09:00:00 to 2018-11-21 09:00:00
Step 5:Downloading data from 2018-11-21 09:00:00 to 2018-11-26 09:00:00
	 Downloaded data of len 7201 from 2018-11-21 09:00:00 to 2018-11-26 09:00:00
Step 6:Downloading data from 2018-11-26 09:00:00 to 2018-12-01 09:00:00
	 Downloaded data of len 720

	 Downloaded data of len 7201 from 2019-07-24 09:00:00 to 2019-07-29 09:00:00
Step 55:Downloading data from 2019-07-29 09:00:00 to 2019-08-03 09:00:00
	 Downloaded data of len 7201 from 2019-07-29 09:00:00 to 2019-08-03 09:00:00
Step 56:Downloading data from 2019-08-03 09:00:00 to 2019-08-08 09:00:00
	 Downloaded data of len 7201 from 2019-08-03 09:00:00 to 2019-08-08 09:00:00
Step 57:Downloading data from 2019-08-08 09:00:00 to 2019-08-13 09:00:00
	 Downloaded data of len 7201 from 2019-08-08 09:00:00 to 2019-08-13 09:00:00
Step 58:Downloading data from 2019-08-13 09:00:00 to 2019-08-18 09:00:00
	 Downloaded data of len 6721 from 2019-08-13 09:00:00 to 2019-08-18 09:00:00
Step 59:Downloading data from 2019-08-18 09:00:00 to 2019-08-23 09:00:00
	 Downloaded data of len 7201 from 2019-08-18 09:00:00 to 2019-08-23 09:00:00
Step 60:Downloading data from 2019-08-23 09:00:00 to 2019-08-28 09:00:00
	 Downloaded data of len 7201 from 2019-08-23 09:00:00 to 2019-08-28 09:00:00
Step 61:Download

Step 109:Downloading data from 2020-04-24 09:00:00 to 2020-04-29 09:00:00
	 Downloaded data of len 7051 from 2020-04-24 09:00:00 to 2020-04-29 09:00:00
Step 110:Downloading data from 2020-04-29 09:00:00 to 2020-05-04 09:00:00
	 Downloaded data of len 7201 from 2020-04-29 09:00:00 to 2020-05-04 09:00:00
Step 111:Downloading data from 2020-05-04 09:00:00 to 2020-05-09 09:00:00
	 Downloaded data of len 7201 from 2020-05-04 09:00:00 to 2020-05-09 09:00:00
Step 112:Downloading data from 2020-05-09 09:00:00 to 2020-05-14 09:00:00
	 Downloaded data of len 7201 from 2020-05-09 09:00:00 to 2020-05-14 09:00:00
Step 113:Downloading data from 2020-05-14 09:00:00 to 2020-05-19 09:00:00
	 Downloaded data of len 7201 from 2020-05-14 09:00:00 to 2020-05-19 09:00:00
Step 114:Downloading data from 2020-05-19 09:00:00 to 2020-05-24 09:00:00
	 Downloaded data of len 7201 from 2020-05-19 09:00:00 to 2020-05-24 09:00:00
Step 115:Downloading data from 2020-05-24 09:00:00 to 2020-05-29 09:00:00
	 Downloaded d

Step 163:Downloading data from 2021-01-19 09:00:00 to 2021-01-24 09:00:00
	 Downloaded data of len 7201 from 2021-01-19 09:00:00 to 2021-01-24 09:00:00
Step 164:Downloading data from 2021-01-24 09:00:00 to 2021-01-29 09:00:00
	 Downloaded data of len 7201 from 2021-01-24 09:00:00 to 2021-01-29 09:00:00
Step 165:Downloading data from 2021-01-29 09:00:00 to 2021-02-03 09:00:00
	 Downloaded data of len 7201 from 2021-01-29 09:00:00 to 2021-02-03 09:00:00


In [33]:
#from_date = '2019-06-01 00:00:00'
#to_date = '2020-01-01 00:00:00'
from_date = '2020-09-22 00:00:00'
to_date = '2021-02-06 00:00:00'
output_filename = 'AVAXUSDT_1minute_kline'
step = 5
pause = 5
symbol = 'AVAXUSDT'

In [34]:
download_data_from_binance(symbol, client,from_date, to_date, output_filename, step, pause, simulate=False)

Step 0:Downloading data from 2020-09-22 09:00:00 to 2020-09-27 09:00:00
	 Downloaded data of len 7201 from 2020-09-22 09:00:00 to 2020-09-27 09:00:00
Step 1:Downloading data from 2020-09-27 09:00:00 to 2020-10-02 09:00:00
	 Downloaded data of len 7201 from 2020-09-27 09:00:00 to 2020-10-02 09:00:00
Step 2:Downloading data from 2020-10-02 09:00:00 to 2020-10-07 09:00:00
	 Downloaded data of len 7201 from 2020-10-02 09:00:00 to 2020-10-07 09:00:00
Step 3:Downloading data from 2020-10-07 09:00:00 to 2020-10-12 09:00:00
	 Downloaded data of len 7201 from 2020-10-07 09:00:00 to 2020-10-12 09:00:00
Step 4:Downloading data from 2020-10-12 09:00:00 to 2020-10-17 09:00:00
	 Downloaded data of len 7201 from 2020-10-12 09:00:00 to 2020-10-17 09:00:00
Step 5:Downloading data from 2020-10-17 09:00:00 to 2020-10-22 09:00:00
	 Downloaded data of len 7201 from 2020-10-17 09:00:00 to 2020-10-22 09:00:00
Step 6:Downloading data from 2020-10-22 09:00:00 to 2020-10-27 09:00:00
	 Downloaded data of len 720

In [35]:
#from_date = '2019-06-01 00:00:00'
#to_date = '2020-01-01 00:00:00'
from_date = '2019-07-05 00:00:00'
to_date = '2021-02-06 00:00:00'
output_filename = 'DOGEUSDT_1minute_kline'
step = 5
pause = 5
symbol = 'DOGEUSDT'

In [36]:
download_data_from_binance(symbol, client,from_date, to_date, output_filename, step, pause, simulate=False)

Step 0:Downloading data from 2019-07-05 09:00:00 to 2019-07-10 09:00:00
	 Downloaded data of len 7021 from 2019-07-05 09:00:00 to 2019-07-10 09:00:00
Step 1:Downloading data from 2019-07-10 09:00:00 to 2019-07-15 09:00:00
	 Downloaded data of len 7201 from 2019-07-10 09:00:00 to 2019-07-15 09:00:00
Step 2:Downloading data from 2019-07-15 09:00:00 to 2019-07-20 09:00:00
	 Downloaded data of len 7201 from 2019-07-15 09:00:00 to 2019-07-20 09:00:00
Step 3:Downloading data from 2019-07-20 09:00:00 to 2019-07-25 09:00:00
	 Downloaded data of len 7201 from 2019-07-20 09:00:00 to 2019-07-25 09:00:00
Step 4:Downloading data from 2019-07-25 09:00:00 to 2019-07-30 09:00:00
	 Downloaded data of len 7201 from 2019-07-25 09:00:00 to 2019-07-30 09:00:00
Step 5:Downloading data from 2019-07-30 09:00:00 to 2019-08-04 09:00:00
	 Downloaded data of len 7201 from 2019-07-30 09:00:00 to 2019-08-04 09:00:00
Step 6:Downloading data from 2019-08-04 09:00:00 to 2019-08-09 09:00:00
	 Downloaded data of len 720

	 Downloaded data of len 7201 from 2020-03-31 09:00:00 to 2020-04-05 09:00:00
Step 55:Downloading data from 2020-04-05 09:00:00 to 2020-04-10 09:00:00
	 Downloaded data of len 7201 from 2020-04-05 09:00:00 to 2020-04-10 09:00:00
Step 56:Downloading data from 2020-04-10 09:00:00 to 2020-04-15 09:00:00
	 Downloaded data of len 7201 from 2020-04-10 09:00:00 to 2020-04-15 09:00:00
Step 57:Downloading data from 2020-04-15 09:00:00 to 2020-04-20 09:00:00
	 Downloaded data of len 7201 from 2020-04-15 09:00:00 to 2020-04-20 09:00:00
Step 58:Downloading data from 2020-04-20 09:00:00 to 2020-04-25 09:00:00
	 Downloaded data of len 7051 from 2020-04-20 09:00:00 to 2020-04-25 09:00:00
Step 59:Downloading data from 2020-04-25 09:00:00 to 2020-04-30 09:00:00
	 Downloaded data of len 7201 from 2020-04-25 09:00:00 to 2020-04-30 09:00:00
Step 60:Downloading data from 2020-04-30 09:00:00 to 2020-05-05 09:00:00
	 Downloaded data of len 7201 from 2020-04-30 09:00:00 to 2020-05-05 09:00:00
Step 61:Download

ConnectionError: ('Connection aborted.', OSError("(104, 'ECONNRESET')",))

In [8]:
coinmarketcap_api = '7d201cbc-48cf-48bd-8d72-3540c55a2aa0'

In [9]:
#This example uses Python 2.7 and the python-request library.

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': coinmarketcap_api,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
coins_of_interest = [x['symbol'] for x in data['data']][:100]

In [12]:
coins_of_interest

['BTC',
 'ETH',
 'USDT',
 'BNB',
 'USDC',
 'XRP',
 'ADA',
 'SOL',
 'LUNA',
 'BUSD',
 'AVAX',
 'DOGE',
 'DOT',
 'SHIB',
 'UST',
 'MATIC',
 'DAI',
 'CRO',
 'WBTC',
 'LTC',
 'ATOM',
 'LINK',
 'TRX',
 'LEO',
 'FTT',
 'BCH',
 'UNI',
 'NEAR',
 'ALGO',
 'MANA',
 'XLM',
 'HBAR',
 'FTM',
 'BTCB',
 'ICP',
 'ETC',
 'KLAY',
 'SAND',
 'EGLD',
 'FIL',
 'AXS',
 'VET',
 'XMR',
 'THETA',
 'XTZ',
 'HNT',
 'EOS',
 'MIOTA',
 'CAKE',
 'AAVE',
 'FLOW',
 'BTT',
 'MKR',
 'GRT',
 'BSV',
 'GALA',
 'ONE',
 'KCS',
 'STX',
 'TUSD',
 'HT',
 'NEO',
 'ZEC',
 'XEC',
 'QNT',
 'ENJ',
 'RUNE',
 'AMP',
 'NEXO',
 'KDA',
 'KSM',
 'CRV',
 'WAVES',
 'OKB',
 'CVX',
 'CHZ',
 'LRC',
 'BAT',
 'CELO',
 'USDP',
 'DASH',
 'AR',
 'TFUEL',
 'XEM',
 'SCRT',
 'MINA',
 'XYM',
 'BORA',
 'ROSE',
 'DCR',
 'YFI',
 'ANC',
 'CEL',
 'HOT',
 'COMP',
 'IOTX',
 'XDC',
 'RENBTC',
 'GNO',
 'QTUM']

In [22]:
1 + (5/7) + (5/9)

2.2698412698412698

In [23]:
3850/2.26

1703.5398230088497

In [24]:
(5/7)*1700

1214.2857142857142

In [25]:
1703*125/500

425.75

In [26]:
425*500

212500

In [28]:
(5/9)*1703

946.1111111111112

In [ ]:
'BTC', 'ETH','USDT','BNB','USDC','XRP','ADA','SOL','BUSD','AVAX','DOGE','DOT','SHIB','UST','MATIC',

In [ ]:
['BTC',
 'ETH',
 'USDT',
 'ADA',
 'DOT',
 'XRP',
 'BNB',
 'LINK',
 'LTC',
 'DOGE',
 'XLM',
 'BCH',
 'USDC',
 'AAVE',
 'UNI',
 'WBTC',
 'BSV',
 'EOS',
 'EGLD',
 'ATOM',
 'XEM',
 'XMR',
 'THETA',
 'TRX',
 'MKR',
 'XTZ',
 'SNX',
 'AVAX',
 'COMP',
 'SOL',
 'VET',
 'SUSHI',
 'DAI',
 'NEO',
 'HT',
 'MIOTA',
 'UMA',
 'LUNA',
 'BUSD',
 'CRO',
 'FTT',
 'LEO',
 'GRT',
 'ALGO',
 'FIL',
 'DASH',
 'CEL',
 'YFI',
 'ZRX',
 'KSM',
 'DCR',
 'ZEC',
 'REV',
 'ETC',
 'NEAR',
 'ZIL',
 'WAVES',
 'NEXO',
 'REN',
 'LRC',
 'CHSB',
 'HBAR',
 'OMG',
 'CRV',
 'RENBTC',
 'RUNE',
 '1INCH',
 'CELO',
 'BAT',
 'ONT',
 'DGB',
 'VGX',
 'BTT',
 'ICX',
 'HUSD',
 'NANO',
 'QNT',
 'QTUM',
 'HEDG',
 'ALPHA',
 'SC',
 'KNC',
 'MATIC',
 'CAKE',
 'RSR',
 'ZEN',
 'OKB',
 'TUSD',
 'MANA',
 'AMPL',
 'FTM',
 'STX',
 'ENJ',
 'OCEAN',
 'IOST',
 'BNT',
 'RVN',
 'XVG',
 'BTCB',
 'EWT']